In [1]:
!pip3 install pygame

     |████████████████████████████████| 11.4MB 2.7MB/s 


In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
device = "cpu" #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: True 


Data description : 
 One array of size 30
 - the result of the remaing dices action : four slots
   The numbers go from 0 to 6 : 0 if already played it
  
 - the board state composed of :
      - 24 first digit from -9 to +9 for black and white pins on the arrows
      - 2 folowing for number of pins that finished : white and black
      - 2 last for number of pins in prison : white and black

The output represents the space of possible actions.
We need to map every possible action to an integer.
We have 25 slots from where a pin can move with all 6 possible dice rolls, which leads to 150 possible actions.  



In [0]:
input_size_ = 32
output_size_ = 175


This function take as an argument an integer from 0 to 149 and returns a tuple (i,j,k) where : 
  - i = arrow id
  - value of dice used
  - -1, 0 or 1 if the pin eats another one, nothing particular, the pin goes out


In [0]:
def compute_action_space():
  table = np.array([[[0]*3]*7]*25)
  for k in range(25):
    for l in range(7):
      table[k][l][0] = k
      table[k][l][1] = l
      if k+l>24:
        table[k][l][2] = 1

action_space = compute_action_space()

def get_action(indx, eats):
  pos, roll, result = action_space[indx//6][indx%6]
  if eats :
    result = -1
  return (pos, roll, result)

def get_idx(action):
  pos, roll, _ = action
  return pos*6+roll


Let's create a functions to help later.

In [0]:
ref_score = 24*15

def get_score(board):
  result = 0
  result+= (board[24]-board[25])*24
  for k,pins in enumerate(board[:24]) :
    if pins > 0 :
      if pins > 1 :
        result += (pins*((k//6)+1)*6+2)
      else :
        result+= ((k//6)+1)
  result/=ref_score
  return result
    

def max_value(resulting_states):
  s = resulting_states[0]
  m = get_score(s.board[:28])
  i=0
  for k,res in enumerate(resulting_states[1:]) :
    sc = get_score(res.board[:28])
    if sc > m :
      m = sc
      s = res
      i=k
  if s.board[24]==15:
    m+=1
  return m,s,i

Now we can use gym and write the data generation loop.

First lets load the necessary files.

We will need a structure to store the data we generate.

In [0]:
class data_manager():
  def __init__(self, df_size):
    self.df = []
    self.df_size_ = df_size

  def reset(self):
    self.df = []

  def is_full(self):
    return len(self.df) == self.df_size_

  def add(self, plays):
    to_add = plays.copy()
    if self.is_full():
      print("already full")
    else :
      for play in to_add :
        self.df.append(play)

  def get_df(self):
    return self.df



In [8]:
!git clone https://github.com/jcGourcuff/BackGammon.git

Cloning into 'BackGammon'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 84 (delta 36), reused 82 (delta 34), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [16]:
%cd BackGammon/

/content/BackGammon


In [17]:
from BackGammon.envs.backgammon_env  import BackgammonEnv
from BackGammon.envs.backgammon import State, WhiteAgent

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [0]:
nb_episodes = 10
Backenv = BackgammonEnv()
max_steps = 1000
epsilon = 1.
epsilon_min = .01
epsilon_decay = .75

In [0]:
def convert_roll(dice_roll):
  roll = dice_roll
  if roll[0]!=roll[1]:
    return roll+ [0,0]
  else : 
    return roll + roll
 

def next_step(agent, state, dice, choice = 'greed'):
  """
  From a board state and a roll, return the best outcome State, its score, the move chosen and its corresponding array.
  """
  state = state
  possible_moves = agent.possible_moves(dice)
  move, next_state, reward = None, None, 0

  if choice == 'greed' :
      possible_next_states = [State(state, move = m) for m in possible_moves]
      score, next_state, idx = max_value(possible_next_states)
      next_state = possible_next_states[idx]
      move = possible_moves[idx]

  else :
      move = possible_moves[np.random.randint(len(possible_moves))]
      next_state = State(previous_state = state, move = move)
      
      
  if next_state.board[24]==15 :
          reward = 1
  Q_value = get_score(next_state.board[:28])-get_score(state.board[:28])

  return next_state, Q_value, reward, move, get_idx(move)


 

def generate_data(episodes, env, max_steps, epsilon, verbose = False):
  eps = epsilon
  data = data_manager(episodes*max_steps)
  episode = 0
  while episode < episodes and not data.is_full():
      # Reset the game state, done and score before every episode/game
      env.reset() #Gets current game state
      done = False        #decides whether the game is over
      mem_state = State()
      k=0
      if verbose : 
        print("start episode "+str(episode))

      while k < max_steps and not data.is_full() : 
        white_roll = [np.random.randint(1, 7), np.random.randint(1, 7)]
        plays = []
        action = []
        rolls = convert_roll(white_roll)
        rolls_input = rolls.copy()

        if verbose :
          print("turn {}".format(k), end = '\r')

        for i,roll in enumerate(rolls) :
          if roll >0 :
            roll_for_dataset = rolls_input.copy()
            if 0.5 > np.random.random():
              roll_for_dataset[0], roll_for_dataset[1] = roll_for_dataset[1],roll_for_dataset[0]
            modelInput =  roll_for_dataset + mem_state.board[:28].copy()
            white = WhiteAgent(mem_state)
            next_state = None
            move = None
            target = None
            score = None

            if np.random.uniform(0, 1) > eps:
              next_state, Q_value, reward, move, target = next_step(white,mem_state, roll)
            else :
              next_state, Q_value, reward, move, target = next_step(white,mem_state, roll, choice = 'random')
            
            finished = (next_state.board[24]==15)|(next_state.board[25]==15)

            potential = 0
            if not finished :
              pot_roll = np.random.randint(1, 7)
              _, potential, _, _ = next_step(WhiteAgent(next_state), next_state, pot_roll)

            else :
              done = True
              
            plays.append([modelInput, move, Q_value, potential, finished, move])

            action.append(move)
            
            mem_state = State(previous_state = mem_state, move = move)
          
          else:
            break

          rolls_input[i]=0

        data.add(plays)
        _, _, _, _ = env.step(action)

        k+=1

        if done:
          episode +=1
          break

      # Reducing our epsilon each episode (Exploration-Exploitation trade-off)
      #if eps >= epsilon_min:
      #    eps *= epsilon_decay

  return data
      

In [0]:
train_data = generate_data(nb_episodes, Backenv, max_steps = max_steps, epsilon = epsilon)

In [0]:
train_data = train_data.get_df()



In [22]:
len(train_data)

544

Now let's create the model.

In [0]:
class BackgammonNet(nn.Module):
  def __init__(self):
    super(BackgammonNet, self).__init__()
    self.input_size = 32
    self.output_size = 175
    self.classifier = nn.Sequential(nn.Linear(self.input_size,self.input_size),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(inplace = False),
                                    nn.Linear(self.input_size,100),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(inplace = False),
                                    nn.Linear(100,200),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(inplace = False),
                                    nn.Linear(200,200),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(inplace = False),
                                    nn.Linear(200,self.output_size))
    
  def forward(self,x):
    x = self.classifier(x)
    return x



Now lets instanciate a torch-compatible dataset.


In [0]:
class PlaysDataset(Dataset):

  def __init__(self, data, transform = None):
      self.data = data

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):

      if torch.is_tensor(idx):
          idx = idx.tolist()

      line = self.data[idx].copy()
      ins = np.array(line[0])
      target = np.array(line[1])
      score = np.array(line[2])
      potential = np.array(line[3])
      finished = np.array(line[4])
      move_tuple = np.array(line[5])
      sample = {'ins': ins, 'target': target, 'score':score, 'potential':potential, 'finished':finished, 'move_tuple':move_tuple}
      return sample


In [0]:
batch_size_ = 10
epochs_ = 10

train_set = PlaysDataset(train_data)
train_loader = DataLoader(train_set, batch_size = batch_size_, shuffle = True, num_workers = 6)

We now need to define our loss function.

In [0]:
alpha = 0.1
gamma = 0.1

mse_loss = nn.MSELoss().to(device)

def custom_loss(,score, potential, is_win, final_move):
  if final_move :
     if is_win :
       return mse_loss(1, torch.floor(potential))
  else :
    return mse_loss(score, )



Let's also choose a learning rate and an optimizer.

In [0]:
loss_fn = custom_loss
learning_rate = 1e-2
model = BackgammonNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
sftmx = nn.functional.log_softmax

Let's write the training loop.

In [0]:
def train(model, optimizer, dataloader, size, learning_rate, num_epoch, train_mode = 'learn_to_play'):
  model.train()
  running_loss = 0.0
  moves = []

  for epoch in range(num_epoch):
    nb_possible = 0.0
    for batch in dataloader :
      
      ins = torch.LongTensor(batch['ins']).type(torch.FloatTensor)
      ins = ins.to(device)

      targets = torch.LongTensor(torch.tensor(batch['target'])).to(device)

      outs = model(ins)
      outs = torch.FloatTensor(sftmx(outs)).to(device)

      scores = torch.DoubleTensor(batch['score']).type(torch.FloatTensor).to(device)
      
      potentials = torch.DoubleTensor(batch['potential']).type(torch.FloatTensor).to(device)

      loss = loss_fn(outs, targets, scores, potentials)

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      _, preds = torch.max(outs, 1)

      l = loss.data.item()
      running_loss += l
      moves = moves + preds.tolist() 
    epoch_loss = running_loss / size
    print('epoch: {} Loss: {:.4f}'.format(epoch, epoch_loss))
  return moves

In [50]:
moves = train(model, optimizer, train_loader,len(train_data), learning_rate, num_epoch= epochs_ )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


epoch: 0 Loss: 0.4058
epoch: 1 Loss: 0.8119
epoch: 2 Loss: 1.2111
epoch: 3 Loss: 1.6096
epoch: 4 Loss: 2.0066
epoch: 5 Loss: 2.4085
epoch: 6 Loss: 2.8033
epoch: 7 Loss: 3.1954
epoch: 8 Loss: 3.5913
epoch: 9 Loss: 3.9822
